# Desafio Frexco

In [1]:
# Importando bibliotecas que seram utilizada para esse desafio

# Importando biblioteca para análise dos dados
import pandas as pd
import numpy as np

# Biblioteca para visualização dos dados
import matplotlib.pyplot as plt

from statsmodels.tsa.ar_model import AutoReg

import seaborn as sns

In [2]:
# Lendo o arquivo com pandas
base_dados = pd.read_excel('Dados.xlsx', engine='openpyxl')

# Visualizando a base de dados
base_dados.head

<bound method NDFrame.head of          Data  Vendas
0  2022-12-06     870
1  2022-12-07     868
2  2022-12-08    1189
3  2022-12-09     742
4  2022-12-10     317
5  2022-12-11     685
6  2022-12-12    1366
7  2022-12-13    1213
8  2022-12-14    1055
9  2022-12-15    1343
10 2022-12-16     832
11 2022-12-17     240
12 2022-12-18     235
13 2022-12-19    1050
14 2022-12-20     711
15 2022-12-21     745
16 2022-12-22    1009
17 2022-12-23      18
18 2022-12-24      40
19 2022-12-25      67
20 2022-12-26     821
21 2022-12-27     572
22 2022-12-28     429
23 2022-12-29     638
24 2022-12-30     106
25 2022-12-31      54
26 2023-01-01     144
27 2023-01-02     814
28 2023-01-03     679
29 2023-01-04     712
30 2023-01-05    1229
31 2023-01-06     821
32 2023-01-07     319
33 2023-01-08     317
34 2023-01-09    1317
35 2023-01-10     807
36 2023-01-11     923
37 2023-01-12    1265
38 2023-01-13     892
39 2023-01-14     289
40 2023-01-15     566
41 2023-01-16    1692
42 2023-01-17    1097
43

In [3]:
# Verificando o tamanho do arquivo
base_dados.shape

(46, 2)

In [4]:
# Verificando se existe campos nulos 
base_dados.isnull().sum()


Data      0
Vendas    0
dtype: int64

In [5]:
# Verificando as informações sobre a base de dados
base_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Data    46 non-null     datetime64[ns]
 1   Vendas  46 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 864.0 bytes


In [6]:
# Indexando a coluna Data

base_dados.set_index('Data', inplace=True)



## Realizando a previsão com statsmodels